In [1]:
import pandas as pd
import yfinance as yf

In [14]:
all_stock_data = yf.download(['WMT', 'CCC.WA'], start='2022-06-12', end='2024-06-13')

[*********************100%%**********************]  2 of 2 completed


In [25]:
wmt = yf.Ticker("WMT")
wmt.info['marketCap']

535579099136

In [28]:
ccc = yf.Ticker("CCC.WA")
ccc.info['marketCap']

8746235904

In [15]:
raw_close = pd.DataFrame({'ccc': all_stock_data.Close['CCC.WA'], 'wmt': all_stock_data.Close.WMT})

In [16]:
close = raw_close.interpolate('linear')

In [18]:
import numpy as np
from scipy import stats
from scipy.optimize import fsolve

In [19]:
def system(x, L, r, k, T, t, E, sigma_E):
    sigma, v = x
    d1 = (np.log(v/L) + ((r - k) + sigma ** 2 / 2) * (T - t)) / (sigma * np.sqrt(T - t))
    d2 = (np.log(v/L) + ((r - k) - sigma ** 2 / 2) * (T - t)) / (sigma * np.sqrt(T - t))
    return v * stats.norm.cdf(d1) - L * np.exp(-r * T) * stats.norm.cdf(d2) - E,\
        sigma_E * E - stats.norm.cdf(d1) * sigma * v


def insolvency_probability(L, r, k, T, t, E, sigma_E):
    solution = fsolve(
        lambda x: system(x, L, r, k, T, t, E, sigma_E),
        x0=np.array([0, 0]))
    sigma, v = solution[0], solution[1]
    return stats.norm.cdf(
        (np.log(L/v) - ((r - k) - sigma ** 2 / 2) * (T - t)) / (sigma * np.sqrt(T - t)))

# Stałe